In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from selenium.common.exceptions import TimeoutException
import random
import re

In [28]:
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
#options.add_argument('--headless') #It allows users to run automated scripts in headless mode, meaning that the browser window wouldn’t be visible. 
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(service=service, options=options)

url = 'https://realt.by/belarus/sale/cottages/?page=1'#'https://realt.by/belarus/sale/dachi/?page=1'#"https://realt.by/sale/dachi/?page=1"
headers = ({'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0', 'Accept-Language':'en-US, en;q=0.5'})
driver.get(url);
time.sleep(5)

#try:
#    cookie_ok_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "cookie-banner__container"))) # 'cookie-banner__container'
#    cookie_ok_button.accept()
#except TimeoutException:
#    print("no cookie popup")

#SCROLL_PAUSE_TIME = random.randint(5,8)

full_height = driver.execute_script("return document.body.scrollHeight")
height = 0 
cnt = 0
res = []

while True:
    res = []
    rand = random.randint(5,8)
    #print(cnt)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    r = soup.find_all("div", class_="sm:w-full w-full p-1.5 sm:p-2.5")
    for e in r:
        add = e.find('a')['href']
        if add not in res:
            res.append(add)
    
    with open('data/pars_res22.txt', 'a') as f:
        f.write(','.join(res)+',') 

    # Calculate new scroll height and compare with last scroll height
    new_height = height + 1.2*driver.execute_script("return document.body.clientHeight")
    try:
        if new_height >= full_height and driver.find_element(By.LINK_TEXT, 'Далее'):
            print(cnt)
            driver.find_element(By.LINK_TEXT, 'Далее').click()
            time.sleep(rand)
            full_height = driver.execute_script("return document.body.scrollHeight")
            new_height = 0
            cnt +=1
    except:
        print('last')
        break    
    height = new_height 
    driver.execute_script(f"window.scrollTo(0, {height});")
    time.sleep(rand)
    

driver.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
last


In [2]:
with open('data/pars_res22.txt', 'r') as f:
    res = f.readlines()

res = res[0].split(',')

len(set(res))

13150

In [3]:
df = pd.DataFrame()
no_price = []

def get_item(url):
    url = 'https://realt.by' + url
    page = requests.get(url)
    time.sleep(3)
    if page.status_code != 200:
        print(url, page.status_code)
        return 0
    
    soup = BeautifulSoup(page.content, "html.parser")
    rdict = {'index': int(url.split('/')[-2])}
    r = soup.find_all('section', class_='bg-white flex flex-wrap md:p-6 my-4 rounded-md') 
    
    sections = {'Параметры объекта':10, 'Инфраструктура':10, 'Местоположение':10}
    for i,k in enumerate(r):
        if k.find('h3').text in sections:
            sections[k.find('h3').text] = i 

    try:
        res_1 = r[sections['Параметры объекта']].find("ul", class_="w-full -my-1").find_all('li', class_='relative py-1')
        for li in res_1:
            key = li.find('span', class_='text-basic sm:flex-shrink-0 mr-2')
            value = li.find('p').getText()
            if key.text not in rdict:
                rdict[key.text] = [value.split()]
    except:
        pass

    try:
        res_2 = r[sections['Инфраструктура']].find_all('p')
        for p in res_2:
            key = p.getText()
            rdict[key] = 1
    except:
        pass

    try:
        res_3 = r[sections['Местоположение']].find('ul', class_='w-full mb-0.5 -my-1')
        for li in res_3:
            key = li.find('span', class_='text-basic sm:flex-shrink-0 mr-2')
            if li.find('a'):
                value = li.find('a').contents[0]
            elif li.find('p'):
                value = li.find('p').getText()
            rdict[key.text] = [value.split()]
    except:
        pass

    try:
        res_4 = soup.find('div', class_='bg-white flex flex-wrap md:p-6 rounded-md').find('h2') 
        rdict['Цена'] = res_4.text.strip()
    except:
        print(url,'no price')
        no_price.append(url)
        pass
    return pd.DataFrame(rdict)


In [4]:
for file in ['data/pars_res2.txt']:#,'data/pars_res3.txt','data/pars_res4.txt','data/pars_res5.txt','data/pars_res6.txt','data/pars_res7.txt','data/pars_res8.txt','data/pars_res9.txt']
    with open(file, 'r') as f:
        res = f.readlines()
        res = res[0].split(',')
        
        for i,m in enumerate(list(set(res))):
            try:    
                df = pd.concat([df,get_item(m)], axis=0)
                df.to_csv('data/country_houses_2.csv')
                #print(i)
            except:
                #print(f'exception for {i,m}')
                continue

https://realt.by/brest-region/sale-cottages/object/3207457/ no price
https://realt.by/sale-cottages/object/2806492/ no price
https://realt.by/grodno-region/sale-cottages/object/3050473/ no price
https://realt.by/mogilev-region/sale-cottages/object/3051630/ no price
https://realt.by/sale-cottages/object/3124244/ no price
https://realt.by/vitebsk-region/sale-cottages/object/3177954/ no price
https://realt.by/gomel-region/sale-cottages/object/3214954/ no price
https://realt.by/brest-region/sale-cottages/object/2227420/ no price
https://realt.by/vitebsk-region/sale-cottages/object/3296486/ no price
https://realt.by/gomel-region/sale-cottages/object/3183436/ no price
https://realt.by/sale-cottages/object/3047826/ no price
https://realt.by/brest-region/sale-cottages/object/3268985/ no price
https://realt.by/grodno-region/sale-cottages/object/3186008/ no price
https://realt.by/sale-cottages/object/3149703/ no price
https://realt.by/sale-cottages/object/3292175/ no price
https://realt.by/brest

In [ ]:
def get_district_details(district_name):
    service = Service(executable_path=r'/usr/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') #It allows users to run automated scripts in headless mode, meaning that the browser window wouldn’t be visible. 
    driver = webdriver.Chrome(service=service, options=options)
    url = 'https://google.com'#"https://realt.by/sale/dachi/?page=1"
    headers = ({'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0', 'Accept-Language':'en-US, en;q=0.5'})
    driver.get(url);
    time.sleep(1)
    #input_box = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "textarea")))
    input_box = driver.find_element(By.CSS_SELECTOR, "textarea")
    input_box.send_keys(district_name)
    input_box.send_keys(Keys.ENTER)
    time.sleep(1)

    wiki_link = driver.find_element(By.XPATH,"//a[contains(@href, 'https://ru.wikipedia.org')]")
    wiki_link.click()

    def get_soup(page):
        soup = BeautifulSoup(page, 'html.parser')
        res = soup.find("span", class_='coordinates plainlinks nourlexpansion').find("a").text
        res = re.findall(r"(\d{2}\°\d{2,}\′*\d*).", res)
        res = [x.replace('°','.') for x in res]
        res = [x.replace('′','') for x in res]
        return (res[0],res[1])
    
    res_1 = get_soup(driver.page_source)

    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        res = soup.find_all('tr')
        for i in res:
            if i.find('th') and i.find('th').text == 'Адм. центр':
                wiki = i.find('a').text
        wiki_link2 = driver.find_element(By.LINK_TEXT, wiki)
        wiki_link2.click()
        res_2 = get_soup(driver.page_source)
    except:
        res_2 = res_1
        

    driver.quit()
    
    return [(float(res_1[0]), float(res_1[1])), (float(res_2[0]), float(res_2[1]))]

In [ ]:

#page = requests.get('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D0%BC%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD')
#soup = BeautifulSoup(page.content, 'html.parser')
##res = soup.find('table')
#res = soup.find_all('tr')
#for i in res:
#   if i.find('th') and i.find('th').text == 'Адм. центр':
#        wiki_link2 = i.find('a').text